# Résolution Pb57 de la collection Hock-Schittkowski

Classification dans l'ouvrage : SQR-P1-1

Number of variables : n = 2

Number of constraints : 3 (1 inequality + 2 bounds)

# Modèle

$\left\{  
\begin{array}{l l l l} 
\min f(x) \\ 
\text{s.c.}\\
0.49x_2-x_1x_2-0.09 &\geq 0\\
x_1\geq 0.4,\  x_2 \geq -4
\end{array} \right.$

où $f : x \longmapsto \sum\limits_{i=1}^{44} f_i(x)^2$

avec $f_i(x) = b_i - x_1 - (0.49-x_1)\exp(-x_2(a_i-8)) \text{ pour }i=1,\ldots,44$ 

Point de départ : $x_0 = (0.42,5)$

Solution attendue : $x^* = (0.419952675,1.284845629)$

Fonction objectif à la solution : $f(x^*) =0.02845966972$

# Résolution avec ENLSIP-0.2.0

In [ ]:
include("../src/enlsip_functions.jl")

In [ ]:
n = 2
m = 44
nb_eq = 0 # nombre de contraintes d'égalité
nb_constraints = 3 # nombre de contraintes d'égalité et d'inégalité

### Données d'entrée

In [ ]:
a = [8.,8.,10.,10.,10.,10.,12.,12.,12.,12.,14.,14.,14.,16.,16.,16.,18.,18.,20.,20.,20.,22.,22.,22., 
         24.,24.,24.,26.,26.,26.,28.,28.,30.,30.,30.,32.,32.,34.,36.,36.,38.,38.,40.,42.]

b = [.49,.49,.48,.47,.48,.47,.46,.46,.45,.43,.45,.43,.43,.44,.43,.43,.46,.45,.42,.42,.43,.41,
         .41,.40,.42,.40,.40,.41,.40,.41,.41,.40,.40,.40,.38,.41,.40,.40,.41,.38,.40,.40,.39,.39]

In [ ]:
# Résidus et matrice jacobienne associée

r_i(x::Vector,t::Float64) = x[1] + (0.49 - x[1]) * exp(-x[2]*(t - 8))
res57 = ResidualsEval(0)

function (res57::ResidualsEval)(x::Vector,rx::Vector,J::Matrix)
    # Evaluate the residuals
    if abs(res57.ctrl) == 1
        rx[:] = b - (t::Float64 -> r_i(x,t)).(a)

    # The jacobian is computed numericaly using forward differences
    # ctrl is set to 0
    elseif res57.ctrl == 2 res57.ctrl = 0 end
    return
end

In [ ]:
# Contraintes et matrice jacobienne associée

cons57 = ConstraintsEval(0)

function (cons57::ConstraintsEval)(x::Vector,cx::Vector,A::Matrix)
    # Evaluate the constraints
    if abs(cons57.ctrl) == 1
        cx[:] = [0.49 * x[2] - x[1] * x[2] - 0.09, x[1] - 0.4, x[2] + 4]
    
    # The jacobian is computed anaticaly
    elseif cons57.ctrl == 2
        A[:] = [-x[2] 0.49-x[1];
        1.0 0.0;
        0.0 1.0]
    end
end

# Résolution avec ENLSIP-0.4.0

In [ ]:
include("../src/enlsip_functions.jl")
x0 = [0.42,5.0]
e = eps(Float64)
se = sqrt(e)
enlsip57 = enlsip(x0,res57,cons57,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)

In [ ]:
x1_sol = enlsip57.sol[1]
x2_sol = enlsip57.sol[2]

println("Solution trouvée avec ENLSIP :")
@printf "x_sol = (%.8e, %.9e)\n" x1_sol x2_sol
@printf "f(x_sol) = %.9e\n" enlsip57.obj_value  
println("\nValeurs théoriques visées :")
@printf "x_opt = (%.8e, %.9e)\n" 0.419952675 1.284845629
@printf "f(x_opt) = %.9e" 0.02845966972

# Résolution avec Ipopt

In [ ]:
using JuMP, Ipopt

In [ ]:
model = Model(with_optimizer(Ipopt.Optimizer))
@variable(model, x1, start = 0.42)
@variable(model, x2, start = 5.)

function f_i(x1, x2, t::Float64)
    return x1 + (0.49 - x1) * exp(-x2*(t - 8))
end

function f(x1, x2)
    y = b - (t::Float64 -> f_i(x1,x2,t)).(a)
    return dot(y,y) 
end

JuMP.register(model, :f, 2, f, autodiff=true)

@NLconstraint(model, c1, 0.49*x2 - x1*x2 - 0.09 >= 0)
@constraint(model, x1 >= 0.4)
@constraint(model, x2 >= -4)

@NLobjective(model, Min, f(x1,x2))

JuMP.optimize!(model)

In [ ]:
1.875+0.128

In [ ]:
println("Solution ENLSIP-Julia :")
@printf "x_sol = (%.8e, %.9e)\n" x1_sol x2_sol
@printf "f(x_sol) = %.9e\n" enlsip57.obj_value 
println("\nSolution Ipopt :")
@printf "x_opt = (%.8e, %.9e)\n" JuMP.value(x1) JuMP.value(x2)
@printf "f(x_opt) = %.9e " JuMP.objective_value(model)